# Configuration of Spark Session to Spark Cluster

In [1]:
from pyspark.sql import SparkSession
from operator import add
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.119:7077") \
        .appName("tim-wywiol-A3")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",4)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .config("spark.cores.max", 4)\
        .getOrCreate()
# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/25 11:50:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/02/25 11:50:12 WARN ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental feature.


# Loading Data in SPARK RDD from HDFS

In [2]:
data_frame = spark_session.read\
    .option("header", "true")\
    .csv('hdfs://192.168.2.119:9000/parking-citations.csv')\
    .cache()

# B1

In [3]:
data_frame.show()

+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+------------------+-----------------+----------------------+
|Ticket number|          Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date| VIN|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount| Latitude|Longitude|Agency Description|Color Description|Body Style Description|
+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+------------------+-----------------+----------------------+
|   1103341116|2015-12-21T00:00:...|      1251|    null|       null|            CA|           200304|null|HOND|        PA|  

In [4]:
#schema
data_frame.printSchema()

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Issue time: string (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: string (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: string (nullable = true)
 |-- VIN: string (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Agency Description: string (nullable = true)
 |-- Color Description: string (nullable = true)
 |-- Body Style Description: string (nullable = true)



In [5]:
data_frame.count()

13077724

In [6]:
data_frame.rdd.getNumPartitions()

16

In [7]:
data4 = data_frame.drop('VIN','Latitude', 'Longitude')

In [8]:
data4.orderBy('Fine amount', ascending=False).show()

+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+-------------------+-----------------+----------------------+
|Ticket number|          Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount| Agency Description|Color Description|Body Style Description|
+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+-------------------+-----------------+----------------------+
|   4360599983|2019-11-17T00:00:...|     410.0|    null|       null|            CA|           202007|FORD|        TK|   WT| 12110 CULVER BLVD W| 502A|  51.0|       80.69.4|           PK OVERSIZ|  

In [9]:
data4.filter(data4['Fine amount'] == '1100').show()

+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+--------------------+-----------------+----------------------+
|Ticket number|          Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount|  Agency Description|Color Description|Body Style Description|
+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+--------------------+-----------------+----------------------+
|   4510322972|2020-02-05T00:00:...|      1127|    null|       null|            CA|           202010|HOND|        PA|   GY|  2055 FIGUEROA ST N|00600|    56|     22511.57B|   DP- RO NOT PRESENT

In [10]:
from pyspark.sql.types import DecimalType
data_5= data4.withColumn("Fine amount",data4["Fine amount"].cast(DecimalType()))

In [11]:
data_5.orderBy('Fine amount', ascending=False).show()

+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+--------------------+-----------------+----------------------+
|Ticket number|          Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount|  Agency Description|Color Description|Body Style Description|
+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+--------------------+-----------------+----------------------+
|   4510804966|2020-02-07T00:00:...|       847|    null|       null|            CA|           202007|AUDI|        PA|   BK|  7257 SUNSET BLVD W|00461|    54|     22511.57B|   DP- RO NOT PRESENT

In [12]:
maxfine = data_5.agg({"Fine amount": "max"}).collect()[0]

In [13]:
print(maxfine)

Row(max(Fine amount)=Decimal('1100'))


In [14]:
numberOfMaxFine = data_5.filter(data_5['Fine amount'] == 1100).count()

In [15]:
print("\033[34m",numberOfMaxFine,"\033[0m", 'tickets have the max fine of 1100 money units')

 626  tickets have the max fine of 1100 money units


## B7

In [18]:
data7 = data_5.groupBy('Make').count() \
            .orderBy('count', ascending=False) \
.limit(20)
data7.show()

+----+-------+
|Make|  count|
+----+-------+
|TOYT|2150768|
|HOND|1479996|
|FORD|1116235|
|NISS| 945133|
|CHEV| 892676|
| BMW| 603092|
|MERZ| 543298|
|VOLK| 432030|
|HYUN| 404917|
|DODG| 391686|
|LEXS| 368420|
| KIA| 328155|
|JEEP| 316300|
|AUDI| 255395|
|MAZD| 242344|
|OTHR| 205546|
| GMC| 184889|
|INFI| 174315|
|CHRY| 159948|
|SUBA| 154640|
+----+-------+



## B8


In [ ]:
### release the cores for another application!
#spark_session.stop()